# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [37]:
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [38]:
# import libraries
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import pandas as pd
import pickle
import re

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline, FeatureUnion


from sqlalchemy import create_engine

In [39]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
df = pd.read_sql_table('messages', engine)
print(df.columns)

df = df.dropna(axis = 0, how = 'any') # TODO - SHould not be needed
#print(df.columns)
X = df.message
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

Index(['id', 'message', 'original', 'genre', 'related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')


In [40]:
print(df.tail())

          id                                            message  \
12328  14671  Ghambat: Khairpur: 100s of villages submerged ...   
12329  14673  Ghambat: Sindh: Volunteers needed : @ http://b...   
12330  14675  shikarpur: Volunteers needed, food and medical...   
12331  14676          Dadu: Needs water, food, medicines. boats   
12333  14679  Umer kot: Sindh: Shelter and Camp needed as pe...   

                                                original   genre  related  \
12328  GHAMBAT: KHAIRPUR: 100S GAVON MEIN PAANI MEY J...  direct        0   
12329  Ghambat: Sindh: Razakaroon ki zaroorat hai : @...  direct        0   
12330  shikarpur mein swanyasevak ki avyashaktha he.k...  direct        0   
12331        JO KO KHANA, PAANI, DAWAYIAN, NAOV CHAHIYE.  direct        0   
12333  Umer kot: Sindh: logon ko rehne ke liye ghar,s...  direct        0   

       request  offer  aid_related  medical_help  medical_products  ...  \
12328        0      0            0             0           

### 2. Write a tokenization function to process your text data

In [41]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [42]:
forest = RandomForestClassifier(n_estimators=100, random_state=1)

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('classifier', MultiOutputClassifier(estimator = forest))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [43]:
print(X.shape)
print(Y.shape)

(10153,)
(10153, 36)


In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

pipeline.fit(X_train, y_train)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...            oob_score=False, random_state=1, verbose=0, warm_start=False),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
y_pred = pipeline.predict(X_test)

In [ ]:
# TODO
#print(y_pred.shape)
y_pred_df = pd.DataFrame(y_pred, columns = y_test.columns)
print(y_pred_df['fire'].shape)
print("y_test: ", y_test['fire'].shape)

for column in y_test.columns:
#    print(column.upper())
    #print(len(y_test[column].astype('int32').tolist()))
    #print(type(y_pred[column].astype('int32').tolist()))
    print(classification_report(y_true = y_test[column].astype('int32').tolist(), y_pred = y_pred_df[column].astype('int32').tolist()))

#y_true_tmp = [0, 1, 2, 2, 2]
#y_pred_tmp = [0, 0, 2, 2, 1]
#target_names = ['class 0', 'class 1', 'class 2']
#print(type(y_true_tmp[0]))
#print(classification_report(y_true_tmp, y_pred_tmp))

(2539,)
y_test:  (2539,)
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2536
           1       0.00      0.00      0.00         3

   micro avg       1.00      1.00      1.00      2539
   macro avg       0.50      0.50      0.50      2539
weighted avg       1.00      1.00      1.00      2539

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2538
           1       0.00      0.00      0.00         1

   micro avg       1.00      1.00      1.00      2539
   macro avg       0.50      0.50      0.50      2539
weighted avg       1.00      1.00      1.00      2539

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2539

   micro avg       1.00      1.00      1.00      2539
   macro avg       1.00      1.00      1.00      2539
weighted avg       1.00      1.00      1.00      2539

              precision    recall  f1-score   su

C:\ProgramData\Anaconda3\envs\env_udacity\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2539

   micro avg       1.00      1.00      1.00      2539
   macro avg       1.00      1.00      1.00      2539
weighted avg       1.00      1.00      1.00      2539

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2539

   micro avg       1.00      1.00      1.00      2539
   macro avg       1.00      1.00      1.00      2539
weighted avg       1.00      1.00      1.00      2539

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2539

   micro avg       1.00      1.00      1.00      2539
   macro avg       1.00      1.00      1.00      2539
weighted avg       1.00      1.00      1.00      2539

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2539

   micro avg       1.00      1.00      1.00      2539
   macro avg     

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
# Which parameters are available?
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'classifier', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'classifier__estimator__bootstrap', 'classifier__estimator__class_weight', 'classifier__estimator__criterion', 'classifier__estimator__max_depth', 'classifier__estimator__max_features', 'classifier__estimator__max_leaf_nodes', 'classifier__estimator__min_impurity_decrease', 'classifier__estimator__min_impurity_split', 'classifier__estimator__min_samples_leaf', 'classifier__estimator__min_samples_split', 'classifier__estimator__min_weight_fraction_leaf', 'classifier__estimator__n_estimators', 'classifier__estimator__n_jobs', 'classi

In [ ]:
parameters = {
    # ()'vect', CountVectorizer(tokenizer=tokenize))
    'vect__stop_words': [None, 'english'],
    # ('tfidf', TfidfTransformer())
    'tfidf__use_idf': [True, False],
    
    #('classifier', MultiOutputClassifier(estimator = forest))
    'classifier__estimator__n_estimators': [50,100],
    'classifier__estimator__criterion': ['entropy', 'gini']
}

cv = GridSearchCV(pipeline, parameters)

In [ ]:
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)

C:\ProgramData\Anaconda3\envs\env_udacity\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


In [ ]:
print("Best parameters set found on development set:")
print(cv.best_params_)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred_df = pd.DataFrame(y_pred, columns = y_test.columns)

for column in y_test.columns:
    print(column.upper())
    print(classification_report(y_true = y_test[column].astype('int32').tolist(), y_pred = y_pred_df[column].astype('int32').tolist()))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
# save the model to disk
filename = 'model.pkl'
pickle.dump(cv, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.